Main scrapy function

In [6]:
%pip install crochet
%pip install scrapy
%pip install scrapy-selenium
%pip install undetected-chromedriver
%pip install pandas
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from datetime import datetime
from typing import Iterable
import scrapy
from scrapy_selenium import SeleniumRequest

# Output file formatting
class itemFormat(scrapy.Item):
    
    card_name = scrapy.Field()
    percentage_in_decks = scrapy.Field()
    img = scrapy.Field()
    specials = scrapy.Field()
    price_per_kg = scrapy.Field()
    
    
class edhrecSpider(scrapy.Spider):
    name = 'searcher'
    
    custom_settings = {
        'FEED_FORMAT': 'json',
        'FEED_URI': f'edhrecScrape_{datetime.now().strftime("%Y%m%d%H%M%S")}.json'
    }
    
    def start_requests(self) -> Iterable[scrapy.Request]:
        # lits all urls that the spider will be crawling through
        urls = [ 'https://edhrec.com/commanders/gimli-mournful-avenger' ] # change this link to check through any commanders you want to
        
        # loop through all the links listed within the urls variable
        for url in urls:
            yield SeleniumRequest(url = url, callback = self.parse)
        return super().start_requests()

    def parse(self, response):
        
        for i, quote in enumerate(response.css('div.CardView_cardWrapper__DVSFy')):
            quote_item = itemFormat()
            
            # Extracting product information
            quote_item['card_name'] = quote.css('span.Card_name__Mpa7S::text').get(i).replace("\u00f3","o")
            quote_item['percentage_in_decks'] = quote.css('div.CardLabel_label__iAM7T::text').get().replace("\n", " || synergy ")
            # quote_item['img'] = quote.css('a img.CardImage_border__OcVcj.shadow::attr(src)').get()
            
            yield quote_item

Run the spider to scrape the site


In [5]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
    
process.crawl(edhrecSpider)
process.start()

2024-07-11 16:31:54 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-07-11 16:31:54 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.7, Platform Windows-10-10.0.19045-SP0
2024-07-11 16:31:54 [scrapy.addons] INFO: Enabled addons:
[]
2024-07-11 16:31:54 [py.warnings] WARNING: c:\Users\josia\AppData\Local\Programs\Python\Python312\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

Show final product list

In [4]:
import pandas as pd
dfjson = pd.read_json('edhrecScrape.json')
dfjson

FileNotFoundError: File edhrecScrape.json does not exist

Format the data

In [ ]:
import json

def sort_json_file(file_path):
    # Read the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Sort items based on a specific attribute, such as 'product_name'
    sorted_data = sorted(data, key=lambda x: x['card_name'])

    # Write sorted contents back to the JSON file
    with open(file_path, 'w') as file:
        json.dump(sorted_data, file, indent=4)

# Example usage:
sort_json_file('edhrecScrape.json')


Only get cards that show up from the user specified percentage and above

In [ ]:
whatPercentage = int(input("what percentage of cards do you want to see?"))

In [ ]:
import pandas as pd
import re

# Read data from the JSON file into a pandas DataFrame
df = pd.read_json('edhrecScrape.json')

# Filter the DataFrame based on the percentage condition
filtered_df = df[df['percentage_in_decks'].str.extract(r'(\d+)%', expand=False).astype(float) >= whatPercentage]

# Output the filtered DataFrame
filtered_df.to_json('filtered_data.json', orient='records')
# print(filtered_df.to_dict(orient='records'))

filtered_search = pd.read_json('filtered_data.json')
filtered_search
